## This is a notebook for building dynamic request on SQL survey database in python

### Scope statement:  
In the Data Wrangling with SQL course, we have seen how we could write stored procedure/functions to build dynamic SQL pivot survey answers data in usable format for analysis in the toy database “SurveySample_A19”.  

After a few iterations, we ended up with the following design:
1. A stored function dbo.fn_GetAllSurveyDataSQL() which generates and returns a dynamic SQL query string for extracting the pivoted survey answer data.  
2. A trigger dbo.trg_refreshSurveyView  
a. firing on INSERT, DELETE and UPDATE upon the table dbo.SurveyStructure  
b. executing a CREATE OR ALTER VIEW vw_AllSurveyData AS + the string returned by dbo.fn_GetAllSurveyDataSQL  

With this design, we have enforced an “always fresh” data policy in the viewvw_AllSurveyData.  
As discussed, this solution is “ideal” as it respects the principle of data locality. But it requires to have privileges for creating stored procedures/functions and triggers. If the former may be rare, the latter is often heavily restricted.  

You are now in a scenario where the only databases operations allowed are:  
1. to select data from tables.  
2. to create/alter views.  

You can use programmatic access to the database server via an ODBC library and you have to develop in Python 3.  

Your Python 3 application must accommodate the following requirements:  
1. Gracefully handle the connection to the database server.  
2. Replicate the algorithm of the dbo.fn_GetAllSurveyDataSQL stored function.  
3. Replicate the algorithm of the trigger dbo.trg_refreshSurveyView for creating/altering the view vw_AllSurveyData whenever applicable.  
4. For achieving (3) above, a persistence component (in any format you like: CSV, XML, JSON, etc.), storing the last known surveys’ structures should be in place. It is not acceptable to just recreate the view every time: the trigger behaviour must be replicated.  
5. Of course, extract the “always-fresh” pivoted survey data, in a CSV file, adequately named.  

In terms of allowed libraries and beyond the recommended pyodbc & pandas, you are free to use anything you like, but with this mandatory requirement: your Python application should not require the user to install packages before the run.  
In order to do so, have a look here: https://stackoverflow.com/questions/12332975/installing-python-module-within-code  

#### Enjoy the work !

#### PART0: managing the proper packages

In [2]:
def install():
    import importlib
    import subprocess
    import sys
    import os.path
    from os import path
    global pd, np, pyodbc, path
    try:
        import pandas as pd
        print("{0} is correctly imported".format('pandas'))
    except ImportError:
        print("{0} has to be imported".format('pandas'))            
        subprocess.call([sys.executable, '-m', 'pip', 'install', 'pandas'])
    finally:
        import pandas as pd
        print("Pandas- Done")
    try:
        import numpy as np
        print("{0} is correctly imported".format('numpy'))
    except ImportError:
        print("{0} has to be installeed".format('numpy'))            
        subprocess.call([sys.executable, '-m', 'pip', 'install', 'numpy'])
    finally:
        import numpy as np
        print("numpy- Done")
    try:
        import pyodbc
        print("{0} is correctly imported".format('pyodbc'))
    except ImportError:
        print("{0} has to be installeed".format('pyodbc'))            
        subprocess.call([sys.executable, '-m', 'pip', 'install', 'pyodbc'])
    finally:
        import pyodbc as pyodbc
        print("pyodbc- Done")
        
    print("all environment properly imported")

In [3]:
install()

pandas is correctly imported
Pandas- Done
numpy is correctly imported
numpy- Done
pyodbc is correctly imported
pyodbc- Done
all environment properly imported


### PART1: connection to database  
- first ensure to have loaded the bak file at the right place in mysql server  - DONE
- make a function to connect to the database  - DONE
- make a function to test the connection is OK - simple error management integrated in the function


In [6]:
def connect_database(my_server="LAPTOP-1NPS6A7O", my_database="Survey_Sample_A19"):
    global sql_conn #we create a global variable for sql connection
    #my_string='DRIVER={SQL Server}; SERVER=LAPTOP-1NPS6A7O;DATABASE=Survey_Sample_A19;Trusted_Connection=yes'
    my_string='DRIVER={SQL Server}; SERVER='+my_server+';DATABASE='+my_database+';Trusted_Connection=yes'
    try:
        sql_conn=pyodbc.connect(my_string)
        query = 'SELECT * FROM ['+my_database+'].[dbo].[Answer]'
        df = pd.read_sql(query, sql_conn)
        print(df.head(3))
        print("connection done")
        return sql_conn
    except pyodbc.Error as ex:
        sqlstate = ex.args[0]
        if sqlstate == '28000':
            print("LDAP Connection failed: check password")
        else:
            print("connection failed. Check Server and database")


In [7]:
connect_database()

   QuestionId  SurveyId  UserId  Answer_Value
0           1         1      42             3
1           1         1     296             5
2           1         1    2634             3
connection done


### PART2: Function to get the structure of the survey datafile
- objective is to build the structure of the survey in order to build the final sql query on all surveys
- build the structure of the request to scan the database for getting its last structure - DONE
- create a cursor to fetch the database for which questions are present - DONE
- create a function that builds the final structure of the survey database in Id and questions Id and saves it as 'las_survey_structure.csv'- DONE
- create a function that checks if a survey_structure file exists, if YES, it imports it, if NO it creates one and 

In [8]:
#we create the variables for the final queries
strQueryTemplateForAnswerColumn = ' COALESCE((SELECT a.Answer_Value FROM Answer as a WHERE a.UserId = u.UserId AND a.SurveyId = <SURVEY_ID> AND a.QuestionId = <QUESTION_ID>), -1) AS ANS_Q<QUESTION_ID> ';
strQueryTemplateForNullColumnn = ' NULL AS ANS_Q<QUESTION_ID> '
strQueryTemplateOuterUnionQuery = ' SELECT UserId, <SURVEY_ID> as SurveyId, <DYNAMIC_QUESTION_ANSWERS> FROM [User] as u WHERE EXISTS (SELECT * FROM Answer as a WHERE u.UserId = a.UserId AND a.SurveyId = <SURVEY_ID>)'
strCurrentUnionQueryBlock = ''
strFinalQuery = ''

In [9]:
strQueryTemplateForAnswerColumn

' COALESCE((SELECT a.Answer_Value FROM Answer as a WHERE a.UserId = u.UserId AND a.SurveyId = <SURVEY_ID> AND a.QuestionId = <QUESTION_ID>), -1) AS ANS_Q<QUESTION_ID> '

In [10]:
cursor1=sql_conn.cursor() #we define cursor1 the main cursor to fetch on all Surveys
cursor1.execute("SELECT SurveyId FROM Survey ORDER BY SurveyId")
for surveyId in cursor1.fetchall():
    print (surveyId)

(1, )
(2, )
(3, )


In [11]:
# we define the cursor2 to fetch on each survey which of the questions present
def my_sq_cursor2(currentSurveyId):
    my_string= 'SELECT * FROM ( SELECT SurveyId,QuestionId,1 as InSurvey FROM SurveyStructure WHERE SurveyId ='+str(currentSurveyId)
    my_string=my_string+' UNION SELECT '+str(currentSurveyId) +' as SurveyId, Q.QuestionId, 0 as InSurvey FROM Question as Q'
    my_string=my_string+' WHERE NOT EXISTS(SELECT * FROM SurveyStructure as S WHERE S.SurveyId = '+str(currentSurveyId)
    my_string=my_string+' AND S.QuestionId = Q.QuestionId)) as t ORDER BY QuestionId'
    return my_string

In [12]:
my_sq_cursor2(2) #we test it works in writing the structure of the cursor as sql language

'SELECT * FROM ( SELECT SurveyId,QuestionId,1 as InSurvey FROM SurveyStructure WHERE SurveyId =2 UNION SELECT 2 as SurveyId, Q.QuestionId, 0 as InSurvey FROM Question as Q WHERE NOT EXISTS(SELECT * FROM SurveyStructure as S WHERE S.SurveyId = 2 AND S.QuestionId = Q.QuestionId)) as t ORDER BY QuestionId'

In [13]:
cursor2=sql_conn.cursor()
cursor2.execute(my_sq_cursor2(2))
for currentSurveyIdInQuestion, currentQuestionID, currentInSurvey in cursor2.fetchall():
    print (currentSurveyIdInQuestion, currentQuestionID, currentInSurvey)

2 1 0
2 2 1
2 3 1
2 4 0


In [14]:
#let's build a function that reports the survey structure based on cursor fetching
def survey_structure(my_sql_connection=sql_conn):
    cursor1=my_sql_connection.cursor()
    cursor1.execute("SELECT SurveyId FROM Survey ORDER BY SurveyId")
    my_final_data=[0,0,0]
    for surveyId in cursor1.fetchall():
        my_surveyId=surveyId[0]
        cursor2=my_sql_connection.cursor()
        cursor2.execute(my_sq_cursor2(my_surveyId))
        for currentSurveyIdInQuestion, currentQuestionID, currentInSurvey in cursor2.fetchall():
            my_data=np.array([currentSurveyIdInQuestion, currentQuestionID, currentInSurvey])
            my_final_data=np.vstack((my_final_data,my_data))
    my_survey_structure=pd.DataFrame(my_final_data[1:,:])
    my_survey_structure.columns=['SurveyId','QuestionID','QuestionInSurvey (1=YES,0=NO)']
    my_survey_structure.to_csv('./last_survey_structure.csv', sep=',')
    print("last_survey_structure.csv created in current directory")
    return my_survey_structure

In [15]:
survey_structure()

last_survey_structure.csv created in current directory


,SurveyId,QuestionID,"QuestionInSurvey (1=YES,0=NO)"
0,1,1,1
1,1,2,1
2,1,3,0
3,1,4,0
4,2,1,0
5,2,2,1
6,2,3,1
7,2,4,0
8,3,1,0
9,3,2,0


### PART3: Build in an iterative process the final query based on the survey structure input
- build independent functions for each part of the Query - DONE
- assemble together in a loop with number of surveys and number of questions as inputs - DONE
- create a function which has the survey structure as input - DONE

In [16]:
# we know look at each questions for each survey and collect the data with proper queries built
def build_strColumnsQueryPart(currentQuestionID, currentInSurvey):
    if currentInSurvey == 0 :#CURRENT QUESTION IS NOT IN THE CURRENT SURVEY
        strColumnsQueryPart = strQueryTemplateForNullColumnn.replace('<QUESTION_ID>',str(currentQuestionID))
    else:
        strColumnsQueryPart =strQueryTemplateForAnswerColumn.replace('<QUESTION_ID>',str(currentQuestionID))
    return str(strColumnsQueryPart)

In [17]:
build_strColumnsQueryPart(1,1)

' COALESCE((SELECT a.Answer_Value FROM Answer as a WHERE a.UserId = u.UserId AND a.SurveyId = <SURVEY_ID> AND a.QuestionId = 1), -1) AS ANS_Q1 '

In [18]:
# we build a function that will be used to do string replacement in the query blocks of the surveys
def build_strCurrentUnionQueryBlock(strQueryTemplateOuterUnionQuery,currentSurveyId,strColumnsQueryPart):
    strCurrentUnionQueryBlock=strQueryTemplateOuterUnionQuery.replace('<DYNAMIC_QUESTION_ANSWERS>', str(strColumnsQueryPart))
    strCurrentUnionQueryBlock=strCurrentUnionQueryBlock.replace('<SURVEY_ID>', str(currentSurveyId))
    return str(strCurrentUnionQueryBlock)

In [19]:
build_strCurrentUnionQueryBlock(strQueryTemplateOuterUnionQuery,1,build_strColumnsQueryPart(1,1))

' SELECT UserId, 1 as SurveyId,  COALESCE((SELECT a.Answer_Value FROM Answer as a WHERE a.UserId = u.UserId AND a.SurveyId = 1 AND a.QuestionId = 1), -1) AS ANS_Q1  FROM [User] as u WHERE EXISTS (SELECT * FROM Answer as a WHERE u.UserId = a.UserId AND a.SurveyId = 1)'

In [20]:
#iterative function to build the final query
def build_final_query(my_survey_structure):
    question_id_list=my_survey_structure['QuestionID'].unique()
    max_question=np.max(question_id_list)
    survey_id_list=my_survey_structure['SurveyId'].unique()
    max_survey=np.max(survey_id_list)
    strFinalQuery=''
    for my_survey_id in survey_id_list:
        my_string_1=''
        for my_question_id in question_id_list:
            my_currentInSurvey=my_survey_structure[(my_survey_structure['SurveyId']==my_survey_id)&(my_survey_structure['QuestionID']==my_question_id)].iloc[:,2].values
            my_string_1=my_string_1+build_strColumnsQueryPart(my_question_id,my_currentInSurvey)
            if my_question_id <max_question:
                my_string_1=my_string_1 + ' , '
            else:
                my_string_1=my_string_1
        strFinalQuery=strFinalQuery+build_strCurrentUnionQueryBlock(strQueryTemplateOuterUnionQuery,my_survey_id,my_string_1)
        if my_survey_id <max_survey:
            strFinalQuery=strFinalQuery + ' UNION '
        else:
            strFinalQuery=strFinalQuery
    return strFinalQuery

In [21]:
build_final_query(survey_structure())

last_survey_structure.csv created in current directory


' SELECT UserId, 1 as SurveyId,  COALESCE((SELECT a.Answer_Value FROM Answer as a WHERE a.UserId = u.UserId AND a.SurveyId = 1 AND a.QuestionId = 1), -1) AS ANS_Q1  ,  COALESCE((SELECT a.Answer_Value FROM Answer as a WHERE a.UserId = u.UserId AND a.SurveyId = 1 AND a.QuestionId = 2), -1) AS ANS_Q2  ,  NULL AS ANS_Q3  ,  NULL AS ANS_Q4  FROM [User] as u WHERE EXISTS (SELECT * FROM Answer as a WHERE u.UserId = a.UserId AND a.SurveyId = 1) UNION  SELECT UserId, 2 as SurveyId,  NULL AS ANS_Q1  ,  COALESCE((SELECT a.Answer_Value FROM Answer as a WHERE a.UserId = u.UserId AND a.SurveyId = 2 AND a.QuestionId = 2), -1) AS ANS_Q2  ,  COALESCE((SELECT a.Answer_Value FROM Answer as a WHERE a.UserId = u.UserId AND a.SurveyId = 2 AND a.QuestionId = 3), -1) AS ANS_Q3  ,  NULL AS ANS_Q4  FROM [User] as u WHERE EXISTS (SELECT * FROM Answer as a WHERE u.UserId = a.UserId AND a.SurveyId = 2) UNION  SELECT UserId, 3 as SurveyId,  NULL AS ANS_Q1  ,  NULL AS ANS_Q2  ,  NULL AS ANS_Q3  ,  NULL AS ANS_Q4  FR

In [22]:
#let's try to apply the query
df = pd.read_sql(build_final_query(survey_structure()), sql_conn)

last_survey_structure.csv created in current directory


In [23]:
df.shape

(4997, 6)

### PART4: build everything together as a function
- check that the survey structure exists before calling it in the build_final_query() function - DONE
- import the last_survey_structure.csv before calling the build_final_query() function - DONE
- save the final all_survey_answers in a csv - DONE

In [24]:
def survey_struct_exists():
    if path.exists("./last_survey_structure.csv")==True:
        print("survey structure exists already ")
        my_survey=pd.read_csv("./last_survey_structure.csv",sep=',',index_col=0)
    else:
        print("survey structure being created")
        my_survey=survey_structure()
    return my_survey
        

In [25]:
survey_struct_exists()

survey structure exists already 


,SurveyId,QuestionID,"QuestionInSurvey (1=YES,0=NO)"
0,1,1,1
1,1,2,1
2,1,3,0
3,1,4,0
4,2,1,0
5,2,2,1
6,2,3,1
7,2,4,0
8,3,1,0
9,3,2,0


###### 

In [27]:
def get_AllSurveyData(my_conn):
    my_survey=survey_struct_exists()
    my_final_query=build_final_query(my_survey)
    df = pd.read_sql(my_final_query, my_conn)
    df.to_csv("./AllSurveydata.csv")
    return df
    

In [28]:
get_AllSurveyData(sql_conn)

survey structure exists already 


,UserId,SurveyId,ANS_Q1,ANS_Q2,ANS_Q3,ANS_Q4
0,42,1,3.0,-1.0,NaN,None
1,296,1,5.0,-1.0,NaN,None
2,1793,1,-1.0,6.0,NaN,None
3,2411,1,-1.0,1.0,NaN,None
4,2634,1,3.0,-1.0,NaN,None
...,...,...,...,...,...,...
4992,1019491,3,NaN,NaN,NaN,None
4993,1020147,3,NaN,NaN,NaN,None
4994,1021182,3,NaN,NaN,NaN,None
4995,1021328,3,NaN,NaN,NaN,None


### Part5: now that it works, make it a python script independent
- built as a main function that integrates subfonctions - done
- all sub fonctions coded in same code - done
- we save the query as a text file; if survey structure has not changed it uses it, otherwise it builds a new one,
- proper documentation with comments and parameters quotes - almost done
- exceptions and errors management

In [5]:
#this function checks that user has proper installed and imported packages in its environment
def check_install():
    import importlib
    import subprocess
    import sys
    import os.path
    from os import path
    global pd, np, pyodbc, path, sys
    try:
        import pandas as pd
    except ImportError:
        print("{0} has to be installed".format('pandas'))            
        subprocess.call([sys.executable, '-m', 'pip', 'install', 'pandas'])
    finally:
        import pandas as pd
    try:
        import numpy as np
    except ImportError:
        print("{0} has to be installed".format('numpy'))            
        subprocess.call([sys.executable, '-m', 'pip', 'install', 'numpy'])
    finally:
        import numpy as np
    try:
        import pyodbc
    except ImportError:
        print("{0} has to be installed".format('pyodbc'))            
        subprocess.call([sys.executable, '-m', 'pip', 'install', 'pyodbc'])
    finally:
        import pyodbc as pyodbc    
    print("all environment properly constructed")

#this function connects to the database and manages connection errors    
def connect_database(my_server="LAPTOP-1NPS6A7O", my_database="Survey_Sample_A19"):
    global sql_conn #we create a global variable for sql connection
    #my_string='DRIVER={SQL Server}; SERVER=LAPTOP-1NPS6A7O;DATABASE=Survey_Sample_A19;Trusted_Connection=yes'
    my_string='DRIVER={SQL Server}; SERVER='+my_server+';DATABASE='+my_database+';Trusted_Connection=yes'
    try:
        sql_conn=pyodbc.connect(my_string)
        print("connection succeeded.")
        return sql_conn
    except pyodbc.Error as ex:
        sqlstate = ex.args[0]
        if sqlstate == '28000':
            print("LDAP Connection failed: check password")
        else:
            print("connection failed. error is {0} ".format(str(ex.args)))
        sys.exit('database connection issue') #if no connection possible, we stop the program

# this function defines a SQL command string to be used in cursor2 to fetch on each survey ID which of the questions are present
def my_sq_cursor2(currentSurveyId):
    my_string= 'SELECT * FROM ( SELECT SurveyId,QuestionId,1 as InSurvey FROM SurveyStructure WHERE SurveyId ='+str(currentSurveyId)
    my_string=my_string+' UNION SELECT '+str(currentSurveyId) +' as SurveyId, Q.QuestionId, 0 as InSurvey FROM Question as Q'
    my_string=my_string+' WHERE NOT EXISTS(SELECT * FROM SurveyStructure as S WHERE S.SurveyId = '+str(currentSurveyId)
    my_string=my_string+' AND S.QuestionId = Q.QuestionId)) as t ORDER BY QuestionId'
    return my_string

#this function reports the survey structure based on cursor fetching on the SQL survey database
def survey_structure(my_sql_connection):
    cursor1=my_sql_connection.cursor()
    cursor1.execute("SELECT SurveyId FROM Survey ORDER BY SurveyId")
    my_final_data=[0,0,0]
    for surveyId in cursor1.fetchall():
        my_surveyId=surveyId[0]
        cursor2=my_sql_connection.cursor()
        cursor2.execute(my_sq_cursor2(my_surveyId))
        for currentSurveyIdInQuestion, currentQuestionID, currentInSurvey in cursor2.fetchall():
            my_data=np.array([currentSurveyIdInQuestion, currentQuestionID, currentInSurvey])
            my_final_data=np.vstack((my_final_data,my_data))
    my_survey_structure=pd.DataFrame(my_final_data[1:,:]).astype(int)
    my_survey_structure.columns=['SurveyId','QuestionID','QuestionInSurvey (1=YES,0=NO)']
    return my_survey_structure

# this function builds the SQL string to check questions for each survey and collect the respose data
def build_strColumnsQueryPart(currentQuestionID, currentInSurvey):
    strQueryTemplateForAnswerColumn = ' COALESCE((SELECT a.Answer_Value FROM Answer as a WHERE a.UserId = u.UserId AND a.SurveyId = <SURVEY_ID> AND a.QuestionId = <QUESTION_ID>), -1) AS ANS_Q<QUESTION_ID> ';
    strQueryTemplateForNullColumnn = ' NULL AS ANS_Q<QUESTION_ID> '
    if currentInSurvey == 0 :#CURRENT QUESTION IS NOT IN THE CURRENT SURVEY
        strColumnsQueryPart = strQueryTemplateForNullColumnn.replace('<QUESTION_ID>',str(currentQuestionID))
    else:
        strColumnsQueryPart =strQueryTemplateForAnswerColumn.replace('<QUESTION_ID>',str(currentQuestionID))
    return str(strColumnsQueryPart)

# this function unions the independent queries built for each survey with the build_strColumnsQueryPart() function
def build_strCurrentUnionQueryBlock(currentSurveyId,strColumnsQueryPart):
    strQueryTemplateOuterUnionQuery = ' SELECT UserId, <SURVEY_ID> as SurveyId, <DYNAMIC_QUESTION_ANSWERS> FROM [User] as u WHERE EXISTS (SELECT * FROM Answer as a WHERE u.UserId = a.UserId AND a.SurveyId = <SURVEY_ID>)'
    strCurrentUnionQueryBlock = ''
    strCurrentUnionQueryBlock=strQueryTemplateOuterUnionQuery.replace('<DYNAMIC_QUESTION_ANSWERS>', str(strColumnsQueryPart))
    strCurrentUnionQueryBlock=strCurrentUnionQueryBlock.replace('<SURVEY_ID>', str(currentSurveyId))
    return str(strCurrentUnionQueryBlock)

#this function iterates over the survey structure to build the final query using subfonctions
#build_strColumnsQueryPart() and build_strCurrentUnionQueryBlock()
def build_final_query(my_survey_structure):
    question_id_list=my_survey_structure['QuestionID'].unique()
    max_question=np.max(question_id_list)
    survey_id_list=my_survey_structure['SurveyId'].unique()
    max_survey=np.max(survey_id_list)
    strFinalQuery=''
    for my_survey_id in survey_id_list:
        my_string_1=''
        for my_question_id in question_id_list:
            my_currentInSurvey=my_survey_structure[(my_survey_structure['SurveyId']==my_survey_id)&(my_survey_structure['QuestionID']==my_question_id)].iloc[:,2].values
            my_string_1=my_string_1+build_strColumnsQueryPart(my_question_id,my_currentInSurvey)
            if my_question_id <max_question:
                my_string_1=my_string_1 + ' , '
            else:
                my_string_1=my_string_1
        strFinalQuery=strFinalQuery+build_strCurrentUnionQueryBlock(my_survey_id,my_string_1)
        if my_survey_id <max_survey:
            strFinalQuery=strFinalQuery + ' UNION '
        else:
            strFinalQuery=strFinalQuery
    #we save the final Query in a .txt file
    try:
        with open('./my_query.txt', 'w') as f:
            f.write(strFinalQuery)
        print("final query saved as 'my_query.txt' in working directory")
    except:
        sys.exit("error occured in disk saving final query")


#this function checks that a survey structure has already been stored in the working directory and returns it
# if no survey_structure file exists, it creates and returns it
def survey_struct_exists(my_sql_connection):
    my_output=1
    my_new_survey=survey_structure(my_sql_connection)
    if path.exists("./last_survey_structure.csv")==True:
        print("survey structure exists already. let's check if it has changed.")
        my_old_survey=pd.read_csv("./last_survey_structure.csv",sep=',',index_col=0).astype(int)
        if my_new_survey.equals(my_old_survey)==True:
            print("the survey structure has not changed")
            my_output=1
        else:
            print("the survey structure has changed. We build, save and overwrite the old one.")
            my_output=0
            my_new_survey.to_csv('./last_survey_structure.csv', sep=',')
            print("last_survey_structure.csv properly created in current directory")
    else:
        print("there is no survey structure file saved. We have created it")
        my_new_survey.to_csv('./last_survey_structure.csv', sep=',')
        my_output=0
        
    #then we get or build the final query depending if the survey structure has changed
    if my_output == 0:
        build_final_query(my_new_survey) #we build final query
        with open('./my_query.txt', 'r') as file:
            my_final_query = file.read().replace('\n', '')   
    else:
        with open('./my_query.txt', 'r') as file:
            my_final_query = file.read().replace('\n', '')     
    return my_final_query

def main(my_Server,my_Database):
    print("This is main function running")
    #first we check the packages installed
    check_install()
    
    #second, we connect to database
    my_conn=connect_database(my_Server, my_Database)
    
    #third, we check survey structure and get final query (new built or already existing)
    my_final_query=survey_struct_exists(my_conn)
    
    #then we make the sql query to get the All survey data and save them on "AllSurveydata.csv" file in working directory
    df = pd.read_sql(my_final_query, my_conn)
    df.to_csv("./AllSurveydata.csv")
    print("database sample hereunder")
    print(df.head(3))
    
    print("job done. 'AllSurveydata.csv' saved in working directory")

if __name__ == '__main__':
    theServer=input("what is the server name ? ")
    theDatabase=input("what is the database name ? ")
    main(theServer,theDatabase)

what is the server name ? LAPTOP-1NPS6A7O
what is the database name ? Survey_Sample_A19
This is main function running
all environment properly constructed
connection succeeded.
survey structure exists already. let's check if it has changed.
the survey structure has changed. We build, save and overwrite the old one.
last_survey_structure.csv properly created in current directory
final query saved as 'my_query.txt' in working directory
database sample hereunder
   UserId  SurveyId  ANS_Q1  ANS_Q2  ANS_Q3 ANS_Q4
0      42         1     3.0    -1.0     NaN   None
1     296         1     5.0    -1.0     NaN   None
2    1793         1    -1.0     6.0     NaN   None
job done. 'AllSurveydata.csv' saved in working directory
